In [21]:
using Revise, NeuralNetworkAnalysis, JLD2, SparseArrays

In [46]:
@load "/home/mforets/.julia/dev/NeuralNetworkAnalysis/models/ACC/acc_controller_3_20.jld2" controller;

In [47]:
@taylorize function ACC!(dx, x, p, t)
    local mu = 0.0001 # friction parameter

    # x1 = lead_car position
    # x2 = lead_car velocity
    # x3 = lead_car internal state

    # x4 = ego_car position
    # x5 = ego_car velocity
    # x6 = ego_car internal state
    
    # lead car dynamics
    local a_lead = -2.0

    x_lead, v_lead, γ_lead, x_ego, v_ego, γ_ego, a_ego = x

    dx[1] = v_lead
    dx[2] = γ_lead
    dx[3] = 2 * (a_lead - γ_lead) - mu * v_lead^2
    dx[4] = v_ego
    dx[5] = γ_ego
    dx[6] = 2 * (a_ego - γ_ego) - mu * v_ego^2
    dx[7] = zero(x_lead)
    return dx
end

function relative_values(X)
    v_ego_inf = -ρ(sparsevec([5], [-1.0], 6), X)
    v_ego_sup = ρ(sparsevec([5], [1.0], 6), X)

    # Drel = xlead - xego
    Drel_inf = -ρ(sparsevec([1, 4], [-1.0, 1.0], 6), X)
    Drel_sup = ρ(sparsevec([1, 4], [1.0, -1.0], 6), X)

    # vrel = vlead - vego
    vrel_inf = -ρ(sparsevec([2, 5], [-1.0, 1.0], 6), X)
    vrel_sup = ρ(sparsevec([2, 5], [1.0, -1.0], 6), X)

    low_Y = [Vset, Tgap, v_ego_inf, Drel_inf, vrel_inf]
    high_Y = [Vset, Tgap, v_ego_sup, Drel_sup, vrel_sup]
    Hyperrectangle(low_Y, high_Y)
end

const Vset = 30.0
const Tgap = 1.4

1.4

In [48]:
# define the initial-value problem
X₀ = Hyperrectangle(low=[90., 32, 0, 10, 30, 0],
                    high=[110, 32.2, 0, 11, 30.2, 0])

#X₀ = 0.01 * X₀
#X₀ = overapproximate(X₀, Hyperrectangle)

U₀ = rand(Interval) # ignored
prob = @ivp(x' = ACC!(x), dim: 7, x(0) ∈ X₀ × U₀)
vars_idx = Dict(:state_vars=>1:6, :input_vars=>[], :control_vars=>7:7);

plant = ControlledPlant(prob, controller, vars_idx);
alg = TMJets(abs_tol=1e-9, orderT=8, orderQ=1)
solver = Ai2z()

Ai2z()

In [49]:
# Applying initial control
X0aux = relative_values(X₀)

Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([30.0, 1.4, 30.000000000000004, 79.0, 1.8000000000000007], [30.0, 1.4, 30.2, 100.0, 2.1999999999999993])

In [50]:
radius(X₀)

10.0

In [51]:
radius(X0aux)

100.0

In [52]:
U₀ = forward_network(solver, controller, X0aux)

1-element Array{Zonotope{Float64,Array{Float64,1},Array{Float64,2}},1}:
 Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([27.38906127543748], [-0.7463910167652833 -0.16738928365072347 … 1.591908212326068 -3.811899450144366])

In [53]:
convert(Interval, U₀[1])

Interval{Float64,IntervalArithmetic.Interval{Float64}}([-248.221, 302.999])

-----

In [69]:
@time sol = solve(plant, T=1.0, Tsample=0.1, alg_nn=solver, alg=alg,
                  preprocess=relative_values)

solz = overapproximate(sol, Zonotope);

applying initial control
LinearMap{Float64,CartesianProduct{Float64,Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}},Interval{Float64,IntervalArithmetic.Interval{Float64}}},Float64,SparseMatrixCSC{Float64,Int64}}(
  [1, 1]  =  1.0
  [2, 2]  =  1.0
  [3, 3]  =  1.0
  [4, 4]  =  1.0
  [5, 5]  =  1.0
  [6, 6]  =  1.0, CartesianProduct{Float64,Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}},Interval{Float64,IntervalArithmetic.Interval{Float64}}}(Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([100.0, 32.1, 0.0, 10.5, 30.1, 0.0], [10.0, 0.10000000000000142, 0.0, 0.5, 0.09999999999999787, 0.0]), Interval{Float64,IntervalArithmetic.Interval{Float64}}([-0.873742, 1.0242])))
Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([30.0, 1.4, 30.000000000000004, 79.0, 1.8000000000000007], [30.0, 1.4, 30.2, 100.0, 2.1999999999999993])
Zonotope{Float64,Array{Float64,1},Array{Float64,2}}[Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([27.38906127543748], [-0.7463

In [66]:
H = relative_values(Projection(solz[end][end], 1:6))

Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([30.0, 1.4, -33.69245256031787, 62.21954175917165, -71.22843075237782], [30.0, 1.4, 102.84459608564492, 117.13289638778627, 65.74371413899254])

In [67]:
H.center

5-element Array{Float64,1}:
  30.0
   1.4
 -33.69245256031787
  62.21954175917165
 -71.22843075237782

In [68]:
H.radius

5-element Array{Float64,1}:
  30.0
   1.4
 102.84459608564492
 117.13289638778627
  65.74371413899254

In [ ]:
@show ρ(sparsevec([2], [1.0], 18), solz)
@show -ρ(sparsevec([2], [-1.0], 18), solz)

[radius(solz.F.ext[:controls][i][1]) for i in 1:3]


radius(overapproximate(solz.F.ext[:controls][3][1], Hyperrectangle))